In [1]:
import pandas as pd
import numpy as np
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.porter import *

In [3]:
# impacting the performance by 2 percent why? -- see with 
def preprocess_tweets(tweets):
    '''
    Convert to lowercase
    Remove http tag and urls
    Remove users
    Remove non alpha numeric terms. 
    '''
    tweets['text'] = tweets['text'].str.lower()
    tweets['text'] = tweets['text'].str.replace(r"http\S+", "")
    tweets['text'] = tweets['text'].str.replace(r"http", "")
    tweets['text'] = tweets['text'].str.replace(r"https?://\S+|www\.\S+","")
    tweets['text'] = tweets['text'].str.replace(r"@\S+", "USER")
    #tweets['text'] = tweets['text'].str.replace(r"@", "at")
    tweets['text'] = tweets['text'].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    return tweets
                                                
def remove_html(text):
    html = re.compile('<.*?>')
    return html.sub('',text)
                                                
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

def apply_stemming(tweets):
    stop_words = set(stopwords.words('english'))
    tweets['tokens'] = tweets['text'].apply(wordpunct_tokenize)
    stemmer = PorterStemmer()
    tweets['tokens'] = tweets['tokens'].apply(lambda word:[stemmer.stem(w) for w in word])
    tweets['text'] = tweets["tokens"].str.join(" ")
    tweets.drop(labels = ['tokens'], axis = 1, inplace = True)
    return tweets

In [4]:
# Read the input training file and create the data frame
tweets = pd.read_csv('../Data/train.csv', header = 'infer', delimiter = ',')
tweets.drop(labels = ['id', 'keyword', 'location'], axis = 1, inplace = True)

In [5]:
# Perform text cleaning and preprocessing. 
tweets = preprocess_tweets(tweets)
tweets['text'] = tweets['text'].apply(lambda x: remove_html(x))
tweets['text'] = tweets['text'].apply(lambda x: remove_emoji(x))
tweets['text'] = tweets['text'].apply(lambda x: remove_punct(x))
tweets = apply_stemming(tweets)

In [6]:
tweets

,text,target
0,our deed are the reason of thi earthquak may a...,1
1,forest fire near la rong sask canada,1
2,all resid ask to shelter in place are be notif...,1
3,13000 peopl receiv wildfir evacu order in cali...,1
4,just got sent thi photo from rubi alaska as sm...,1
...,...,...
7608,two giant crane hold a bridg collaps into near...,1
7609,user user the out of control wild fire in cali...,1
7610,m1 94 01 04 utc 5km s of volcano hawaii,1
7611,polic investig after an e bike collid with a c...,1


In [9]:
tweets.to_csv('../Data/clean_train.csv', index = False)

In [10]:
# Pre-Process test set
tweets_test = pd.read_csv('../Data/test.csv', header = 'infer', delimiter = ',')
tweets_test.drop(labels = ['id', 'keyword', 'location'], axis = 1, inplace = True)

tweets_test = preprocess_tweets(tweets_test)
tweets_test['text'] = tweets_test['text'].apply(lambda x: remove_html(x))
tweets_test['text'] = tweets_test['text'].apply(lambda x: remove_emoji(x))
tweets_test['text'] = tweets_test['text'].apply(lambda x: remove_punct(x))
twetweets_testets = apply_stemming(tweets_test)

tweets_test.to_csv('../Data/clean_test.csv', index = False)